# Introduction

Code for [logistic regression blog](https://philliplagoc.wordpress.com/2020/06/24/my-take-on-logistic-regression-part-1/)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
orig_df = pd.read_stata('./hsbdemo.dta')
orig_df.head()

,id,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards,cid
0,45.0,female,low,public,vocation,34.0,35.0,41.0,29.0,26.0,not enrolled,0.0,1
1,108.0,male,middle,public,general,34.0,33.0,41.0,36.0,36.0,not enrolled,0.0,1
2,15.0,male,high,public,vocation,39.0,39.0,44.0,26.0,42.0,not enrolled,0.0,1
3,67.0,male,low,public,vocation,37.0,37.0,42.0,33.0,32.0,not enrolled,0.0,1
4,153.0,male,middle,public,vocation,39.0,31.0,40.0,39.0,51.0,not enrolled,0.0,1


This fake dataset contains variables on 200 students. Our task is to predict `prog` or program type for each student. To simplify the problem, I will use two independent variables: `ses` or socio-economic status, and `write`, or the student's writing score.

- `prog` is the dependent variable. There are three levels: vocation, general, academic
- `ses` is an ordinal independent variable with three levels: low, middle, high
- `write` is a continuous independent variable

I'm going to implement both binary logistic regression and multinomial logistic regression. Thus, there will be two parts to this blog.

Before I implement anything, however, I will preprocess the data by encoding `ses` and `prog`.

### Encoding
Why do we do this? To help our model in reading the data. Machine learning algorithms are built to take in numbers, not strings. So, we have to **encode** the data into numbers. This can be done using **one-hot-encoding**, which creates a boolean variable for each level of `ses`. Pandas' `get_dummies` function handles this for us.

In [5]:
orig_df = pd.get_dummies(orig_df, columns = ['ses'])

For `prog`, I will encode it like so:
- $y = 0$ when the student's program type is general. 
- $y = 1$ when the student's program type is vocational.  
- $y = 2$ when the student's program type is academic. 

In [7]:
orig_df.replace({'prog': {'general': 0, 'vocation': 1, 'academic': 2}}, inplace = True)

Now, let's start with implementing binary logistic regression. To do this, I'll remove all rows where `prog` is academic.

# Binary Logistic Regression 

In [9]:
bi_df = orig_df[['prog', 'ses_low', 'ses_middle', 'ses_high', 'write']]
bi_df = bi_df[bi_df['prog'] != 2]
print(bi_df.shape)
bi_df.head()

(95, 5)


,prog,ses_low,ses_middle,ses_high,write
0,1,1,0,0,35.0
1,0,0,1,0,33.0
2,1,0,0,1,39.0
3,1,1,0,0,37.0
4,1,0,1,0,31.0


### Hypothesis
$$h_{\theta}(x) = \sigma(\theta^Tx) = \frac{1}{1 + e^{-\theta^Tx}}$$

### Cost Function
$$J(\theta) = \frac{1}{m}[\sum_{i = 1}^{m}-y^ilog(h_\theta(x^i))+(1 - y^i)log(1 - h_\theta(x^i))]$$

where $m$ is the number of training samples.

### Deriving Cost Function
$$\frac{\partial{J(\theta)}}{\partial\theta} = \frac{1}{m}(h_\theta(x) - y)x$$

$$\theta_j = \theta_j - \alpha\frac{\partial{J(\theta)}}{\partial\theta}$$

## Implementation from Scratch

## Using Statsmodels

In [17]:
# Exclude ses_low to make this column a baseline. 
# This prevents multicollinearity.
X = bi_df[['ses_middle', 'ses_high', 'write']]

y = bi_df['prog']

bi_model = sm.Logit(y, sm.add_constant(X))
result = bi_model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.649232
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                   prog   No. Observations:                   95
Model:                          Logit   Df Residuals:                       91
Method:                           MLE   Df Model:                            3
Date:                Thu, 25 Jun 2020   Pseudo R-squ.:                 0.06148
Time:                        02:50:08   Log-Likelihood:                -61.677
converged:                       True   LL-Null:                       -65.717
Covariance Type:            nonrobust   LLR p-value:                   0.04437
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2091      1.195      1.849      0.065      -0.133       4.551
ses_middle     0.7408      0.

### Interpreting the results

There's a lot to unpack here, so let's go one by one. 

#### Logit Recap
In linear regression, it was easy to interpret what the coefficients meant. Logistic regression is a bit more difficult, as it requires us to understand what the log-odds are.

Recall from the [first part](https://philliplagoc.wordpress.com/2020/06/24/my-take-on-logistic-regression-part-1/) that the log-odds is computed by:
$$logit(p) = log\frac{p}{1 - p}$$

Again, we are simply taking the log of the odds (log-odds!), which allows us to transform the range from $[0, 1]$, which was what the hypothesis outputs, to a range from $[-\infty, \infty]$. There are other ways to do this transformation e.g., probit, but I'll cover that another time. 

In essence, **logistic regression models the logit-transformed probability as a linear relationship with the independent variables**:

$$logit(p) = log\frac{p}{1 - p} = \beta_0 + \beta_1X_1 + \beta_2X_2$$

Solving for $p$ brings us right back to the sigmoid function!

$$\frac{1-p}{p} = \frac{1}{e^{\beta_0 + \beta_1X_1 + \beta_2X_2}}$$

$$\frac{1}{p} = 1 + \frac{1}{e^{\beta_0 + \beta_1X_1 + \beta_2X_2}}$$

$$\frac{1}{p} = \frac{1 + e^{\beta_0 + \beta_1X_1 + \beta_2X_2}}{e^{\beta_0 + \beta_1X_1 + \beta_2X_2}} = \sigma(\theta^Tx)$$

$$p = \frac{e^{\beta_0 + \beta_1X_1 + \beta_2X_2}}{1 + e^{\beta_0 + \beta_1X_1 + \beta_2X_2}} = \sigma(\theta^Tx)$$

#### What do the coefficients mean?

**Note: [This](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/) article was immensely helpful in understanding what these coefficients meant. Check [this](http://blog.yhat.com/posts/logistic-regression-and-python.html) article, too!**

Each coefficient estimate is the expected change in the log-odds of the student being in the vocational program for each unit increase in the corresponding independent variable, whilst holding the other variables constant.

This means that for every unit increase in the writing score, the log-odds will change by -0.05, and so on for the other variables.

Exponentiating these coefficient estimates will give us the odds of the student being in the vocational program.

In [14]:
print(np.exp(result.params))

const         9.107444
ses_middle    2.097511
ses_high      1.246751
write         0.949620
dtype: float64


For example, we can expect the odds of a student being in the vocational program to **decrease** by about 0.95% for every unit increase in their writing score. 

We can also expect that, if the student has a socioeconomic status categorized as 'middle', the odds of the student being in the vocational program **increases** by about 109%.

# Multinomial Logistic Regression

In [15]:
mn_df = orig_df[['prog', 'ses_low', 'ses_middle', 'ses_high', 'write']]
print(mn_df.shape)
mn_df.head()

(200, 5)


,prog,ses_low,ses_middle,ses_high,write
0,1,1,0,0,35.0
1,0,0,1,0,33.0
2,1,0,0,1,39.0
3,1,1,0,0,37.0
4,1,0,1,0,31.0


### Hypothesis
$$h_{\theta}(x) = \sigma(\theta^Tx) = \frac{1}{1 + e^{-\theta^Tx}}$$

### Cost Function
$$J(\theta) = \frac{1}{m}[\sum_{i = 1}^{m}-y^ilog(h_\theta(x^i))+(1 - y^i)log(1 - h_\theta(x^i))]$$

where $m$ is the number of training samples.

### Deriving Cost Function
$$\frac{\partial{J(\theta)}}{\partial\theta} = \frac{1}{m}(h_\theta(x) - y)x$$

$$\theta_j = \theta_j - \alpha\frac{\partial{J(\theta)}}{\partial\theta}$$

## Implementation from Scratch

## Using Statsmodels

In [18]:
# Exclude ses_low to make this column a baseline. 
# This prevents multicollinearity.
X = mn_df[['ses_middle', 'ses_high', 'write']]

y = mn_df['prog']

mn_model = sm.MNLogit(y, sm.add_constant(X))
result = mn_model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.899909
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                   prog   No. Observations:                  200
Model:                        MNLogit   Df Residuals:                      192
Method:                           MLE   Df Model:                            6
Date:                Thu, 25 Jun 2020   Pseudo R-squ.:                  0.1182
Time:                        02:50:24   Log-Likelihood:                -179.98
converged:                       True   LL-Null:                       -204.10
Covariance Type:            nonrobust   LLR p-value:                 1.063e-08
    prog=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3660      1.174      2.015      0.044       0.065       4.667
ses_middle     0.8247      0.

# Evaluating the results

## Confusion Matrix

## ROC Curve

### Thresholds 